LFU 算法

https://labuladong.github.io/algo/di-yi-zhan-da78c/shou-ba-sh-daeca/suan-fa-ji-fb527/

In [2]:
# Least Frequently Used

In [ ]:
# 第一个 freq_table 以频率 freq 为索引，每个索引存放一个双向链表（注意是每个！）
# 这个链表里存放所有使用频率为 freq 的缓存，缓存里存放三个信息，分别为键 key，值 value，以及使用频率 freq

# 第二个 key_table 以键值 key 为索引，每个索引存放对应缓存在 freq_table 中链表里的内存地址

<img src="460.png" style="zoom:50%" />

In [ ]:
# 对于删除操作
# 因为我们保证了链表中从链表头到链表尾的插入时间是有序的
# 所以 freq_table[minFreq] 的链表中链表尾的节点即为使用频率最小，且插入时间最早的节点
# 我们删除它同时根据情况更新 minFreq

<img src="460(2).png" style="zoom:50%" />

In [ ]:
# 460. LFU 缓存
# https://leetcode.cn/problems/lfu-cache/

class Node:
    def __init__(self, key, val, pre=None, nex=None, freq=0):
        self.pre = pre
        self.nex = nex
        self.freq = freq
        self.val = val
        self.key = key
        
    def insert(self, nex):
        nex.pre = self
        nex.nex = self.nex
        self.nex.pre = nex
        self.nex = nex
    
def create_linked_list():
    head = Node(0, 0)
    tail = Node(0, 0)
    head.nex = tail
    tail.pre = head
    return (head, tail)

class LFUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.size = 0
        self.minFreq = 0
        self.freqMap = collections.defaultdict(create_linked_list)
        self.keyMap = {}

    def delete(self, node):
        if node.pre:
            node.pre.nex = node.nex
            node.nex.pre = node.pre
            if node.pre is self.freqMap[node.freq][0] and node.nex is self.freqMap[node.freq][-1]:
                self.freqMap.pop(node.freq)
        return node.key
        
    def increase(self, node):
        node.freq += 1
        self.delete(node)
        self.freqMap[node.freq][-1].pre.insert(node)
        if node.freq == 1:
            self.minFreq = 1
        elif self.minFreq == node.freq - 1:
            head, tail = self.freqMap[node.freq - 1]
            if head.nex is tail:
                self.minFreq = node.freq

    def get(self, key: int) -> int:
        if key in self.keyMap:
            self.increase(self.keyMap[key])
            return self.keyMap[key].val
        return -1

    def put(self, key: int, value: int) -> None:
        if self.capacity != 0:
            if key in self.keyMap:
                node = self.keyMap[key]
                node.val = value
            else:
                node = Node(key, value)
                self.keyMap[key] = node
                self.size += 1
            if self.size > self.capacity:
                self.size -= 1
                deleted = self.delete(self.freqMap[self.minFreq][0].nex)
                self.keyMap.pop(deleted)
            self.increase(node)


# Your LFUCache object will be instantiated and called as such:
# obj = LFUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

In [ ]:
# 460. LFU 缓存
# https://leetcode.cn/problems/lfu-cache/

class LFUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.keyToVal = {}
        self.keyToFreq = {}
        self.freqToKeys = {}
        self.minFreq = 0

    def get(self, key: int) -> int:
        if key not in self.keyToVal:
            return -1
        self.increaseFreq(key)
        return self.keyToVal[key]

    def put(self, key: int, value: int) -> None:
        if self.capacity == 0:
            return
        if key in self.keyToVal:
            self.keyToVal[key] = value
            self.increaseFreq(key)
            return
        if self.capacity == len(self.keyToVal):
            self.removeMinFreqKey()
        self.keyToVal[key] = value
        self.keyToFreq[key] = 1
        self.freqToKeys.setdefault(1, set()).add(key)
        # 首先，self.freqToKeys是一个defaultdict(set)，它是一个特殊的字典
        # 当访问不存在的键时，会自动创建一个默认值
        # 接下来，.setdefault(1, set())方法会检查键1是否存在于self.freqToKeys字典中
        # 如果存在，则直接返回键1对应的值（即一个空的集合）
        # 如果不存在，则创建一个默认值为set()的键1
        # 最后，.add(key)将key添加到键1对应的集合中，无论是新创建的集合还是已存在的集合
        # 综上所述，self.freqToKeys.setdefault(1, set()).add(key)的目的
        # 是将key添加到self.freqToKeys字典中键为1的对应集合中
        # 如果键1不存在，则会自动创建一个对应的集合并将key添加到集合中
        self.minFreq = 1

    def increaseFreq(self, key: int):
        self.freq = self.keyToFreq[key]
        self.keyToFreq[key] = self.freq + 1
        self.freqToKeys[self.freq].remove(key)
        self.freqToKeys.setdefault(self.freq+1, set()).add(key)
        if len(self.freqToKeys[self.freq]) == 0:
            del self.freqToKeys[self.freq]
            if self.freq == self.minFreq:
                self.minFreq += 1

    def removeMinFreqKey(self):
        keyList = self.freqToKeys[self.minFreq]
        deletedKey = next(iter(keyList))
        keyList.remove(deletedKey)
        if len(keyList) == 0:
            del self.freqToKeys[self.minFreq]


# Your LFUCache object will be instantiated and called as such:
# obj = LFUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

In [ ]:
# setdefault 如果不存在会在原字典里添加一个 key:default_value 并返回 default_value
# get 找不到 key 的时候不会修改原字典，只返回 default_value

In [ ]:
# 删除某个键key肯定是要同时修改三个映射表的，借助minFreq参数可以从FK表中找到freq最小的keyList
# 根据时序，其中第一个元素就是要被淘汰的deletedKey，操作三个映射表删除这个key即可
# 但是有个细节问题，如果keyList中只有一个元素，那么删除之后minFreq对应的key列表就为空了
# 也就是minFreq变量需要被更新。如何计算当前的minFreq是多少呢？

# 实际上没办法快速计算minFreq，只能线性遍历FK表或者KF表来计算，这样肯定不能保证 O(1) 的时间复杂度
# 但是，其实这里没必要更新minFreq变量，因为你想想removeMinFreqKey这个函数是在什么时候调用？
# 在put方法中插入新key时可能调用。而你回头看put的代码，插入新key时一定会把minFreq更新成 1
# 所以说即便这里minFreq变了，我们也不需要管它